In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
# The token can be used to access the repository
!git clone https://github.com/cariello1/BIO-NER-.git

Cloning into 'BIO-NER-'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 493 (delta 75), reused 0 (delta 0), pack-reused 365
Receiving objects: 100% (493/493), 12.49 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [ ]:
%cd /content/BIO-NER-

/content/BIO-NER-


In [ ]:
from preprocessing_1 import PreprocessingDocsCRF
from preprocessing_1 import read_data

# PREPROCESSING

In [ ]:
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
from matplotlib import pyplot as plt
import tqdm

In [ ]:
# Original data in IOB format
file1 = "/content/gdrive/MyDrive/BIO-NER-project/BC4CHEMD-IOB/BC4CHEMD/train.tsv"
file2 = "/content/gdrive/MyDrive/BIO-NER-project/BC4CHEMD-IOB/BC4CHEMD/test.tsv"

In [ ]:
input_train = read_data(file1)
input_test = read_data(file2)

Total number of sentences: 30682 
Total number of sentences: 26364 


In [ ]:
docs_train = PreprocessingDocsCRF(input_train)
data_train_withPOS = docs_train.addPOStags()

1it [00:00,  7.60it/s]

Adding part-of-speech tags to the original corpus (word, label)...


30682it [00:51, 596.87it/s]

Done.


In [ ]:
docs_test = PreprocessingDocsCRF(input_test)
data_test_withPOS = docs_test.addPOStags()

0it [00:00, ?it/s]

Adding part-of-speech tags to the original corpus (word, label)...


26364it [00:45, 584.50it/s]

Done.


In [ ]:
# First sentence of train dataset with word, label, POStag
print('This is what a sentence looks like:{}'.format(data_train_withPOS[0]))

This is what a sentence looks like:[('DPP6', 'NOUN', 'O'), ('as', 'ADP', 'O'), ('a', 'DET', 'O'), ('candidate', 'NOUN', 'O'), ('gene', 'NOUN', 'O'), ('for', 'ADP', 'O'), ('neuroleptic', 'ADJ', 'O'), ('-', '.', 'O'), ('induced', 'ADJ', 'O'), ('tardive', 'ADJ', 'O'), ('dyskinesia', 'NOUN', 'O'), ('.', '.', 'O')]


# FEATURES SET

Morphological features: 

*    Is-Capitalized, Is-Title, Contains-Digits, Contains-Punctuations (Boolean)
*   Prefixes, Suffixes (Nominal)
*   Word's length, Term Frequency (Numeric)

Lexical features: 

*   POS Tagging (Nominal)

Some information from **nearby words** is added, as well as all these features for words that are not at the beginning of a document (BOS), all these features for words that are not at the end of a document (EOS).

Contextual information (-2/+2 token window):

*   Is-Title, Is-Capitalized, Contains-Digits, Contains-Punctuations, Word's length, POS Tagging



In [ ]:
import string 
punct = string.punctuation
def hasPunct(string):
  return any(char for char in string if char in punct)

In [ ]:
def hasNumbers(string):
  return any(char.isdigit() for char in string)

In [ ]:
from collections import Counter
word_freq = Counter(docs_train.tokens)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
word_freq_purify = {key:value for (key,value) in word_freq.items() if key.lower() not in stopwords.words('english') }

In [ ]:
def word_freq(word):
  try: 
    value = word_freq_purify["word"]
  except:
    value = 0
  return value

In [ ]:
 def word2features(sent, i):
    """Map words to dict of features. Code from sklearn-crfsuite API documentation."""
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'wordID': word, #WordIdentity
        'word[-3:]': word[-3:], #suffix
        'word[-2:]': word[-2:],
        'word[:3]': word[:3], #prefix
        'word[:2]': word[:2],
        'word.isupper()': word.isupper(), #isCapitalized
        'hasPunct(word)': hasPunct(word),
        'hasNumber(word)': hasNumbers(word), #ContainsDigits
        'word.istitle()': word.istitle(),
        'len(word)': len(word), #word's length
        'word_freq(word)': word_freq(word), #term frequency
        'postag': postag
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:wordID': word1,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:hasPunct(word)': hasPunct(word),
            '-1:hasNumber(word)': hasNumbers(word),
            '-1:postag': postag1,
            '-1:len(word)': len(word)
        })

    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        features.update({
            '-2:wordID': word1,
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:hasPunct(word)': hasPunct(word),
            '-2:hasNumber(word)': hasNumbers(word),
            '-2:postag': postag1,
            '-2:len(word)': len(word)    
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:wordID': word1,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:hasPunct(word)': hasPunct(word),
            '+1:hasNumber(word)': hasNumbers(word),
            '+1:postag': postag1,
            '+1:len(word)': len(word) 
        })

    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        features.update({
            '+2:wordID': word1,
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:hasPunct(word)': hasPunct(word),
            '+2:hasNumber(word)': hasNumbers(word),
            '+2:postag': postag1,
            '+2:len(word)': len(word) 
        })
    else:
        features['EOS'] = True

    return features

In [ ]:
def sent2features(sent):
    """Map sent to dict of features for each word. Code from sklearn-crfsuite API documentation."""
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
#from preprocessing_1 import sent2features
# Transform a list of sentences in a list of features for each word in a sentence
X_train = [sent2features(s) for s in data_train_withPOS]
X_test = [sent2features(s) for s in data_test_withPOS]

In [ ]:
# Feature dict for the second word of the first sent in test set
X_test[0][2]

{'+1:hasNumber(word)': False,
 '+1:hasPunct(word)': False,
 '+1:len(word)': 15,
 '+1:postag': 'NOUN',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:wordID': 'acid',
 '+2:hasNumber(word)': False,
 '+2:hasPunct(word)': False,
 '+2:len(word)': 15,
 '+2:postag': 'CONJ',
 '+2:word.istitle()': False,
 '+2:word.isupper()': False,
 '+2:wordID': 'and',
 '-1:hasNumber(word)': False,
 '-1:hasPunct(word)': False,
 '-1:len(word)': 15,
 '-1:postag': 'ADP',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:wordID': 'of',
 '-2:hasNumber(word)': False,
 '-2:hasPunct(word)': False,
 '-2:len(word)': 15,
 '-2:postag': 'NOUN',
 '-2:word.istitle()': True,
 '-2:word.isupper()': False,
 '-2:wordID': 'Effects',
 'bias': 1.0,
 'hasNumber(word)': False,
 'hasPunct(word)': False,
 'len(word)': 15,
 'postag': 'NOUN',
 'word.istitle()': False,
 'word.isupper()': False,
 'wordID': 'docosahexaenoic',
 'word[-2:]': 'ic',
 'word[-3:]': 'oic',
 'word[:2]': 'do',
 'word[:3]': 'doc',
 'word_f

In [ ]:
# Transform a list of lbel sentences in a list of labels for each word in a sentence
y_train = [[l for w, pos, l in s] for s in data_train_withPOS]
y_test = [[l for w, pos, l in s] for s in data_test_withPOS]

In [ ]:
y_test[0]

['O',
 'O',
 'B-Chemical',
 'I-Chemical',
 'O',
 'B-Chemical',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

# TRAINING

In [ ]:
%%capture
# python-crfsuite wrapper compatible with sklearn
!pip install sklearn_crfsuite 

In [ ]:
from sklearn_crfsuite import CRF

In [ ]:
crf_model = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)

In [ ]:
%%time
crf_model.fit(X_train, y_train)

CPU times: user 2min 26s, sys: 1.29 s, total: 2min 27s
Wall time: 2min 27s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

# PREDICTION

In [ ]:
%%time
y_hat = crf_model.predict(X_test)

CPU times: user 18.4 s, sys: 132 ms, total: 18.5 s
Wall time: 18.5 s


# REPORT

In [ ]:
%%capture
!pip install nervaluate

In [ ]:
from nervaluate import Evaluator
golden_tags = ["Chemical"]
evaluator = Evaluator(y_test, y_hat, tags = golden_tags, loader="list")
results, results_by_tag = evaluator.evaluate()
import pandas as pd
res = pd.DataFrame.from_dict(results)

In [ ]:
res

,ent_type,partial,strict,exact
correct,20364.000000,18603.000000,18603.000000,18603.000000
incorrect,0.000000,0.000000,1761.000000,1761.000000
partial,0.000000,1761.000000,0.000000,0.000000
missed,4982.000000,4982.000000,4982.000000,4982.000000
spurious,1301.000000,1301.000000,1301.000000,1301.000000
possible,25346.000000,25346.000000,25346.000000,25346.000000
actual,21665.000000,21665.000000,21665.000000,21665.000000
precision,0.939949,0.899308,0.858666,0.858666
recall,0.803440,0.768701,0.733962,0.733962
f1,0.866350,0.828891,0.791432,0.791432


In [ ]:
!pip install seqeval

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=13040911e7139501ac09c9b7c24293eee46023e937cd481222e87459bd5b6d20
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
from seqeval.metrics import f1_score, classification_report

In [ ]:
from seqeval.scheme import IOB2
print('\033[1m' + "Classification report for simple linear CRF model (exact match): ")
print('\033[0m')
print(classification_report(y_test, y_hat,scheme=IOB2, mode="strict"))

Classification report for simple linear CRF model (exact match): 

              precision    recall  f1-score   support

    Chemical       0.86      0.73      0.79     25346

   micro avg       0.86      0.73      0.79     25346
   macro avg       0.86      0.73      0.79     25346
weighted avg       0.86      0.73      0.79     25346



In [ ]:
print("F1-score: {:0.3}".format(f1_score(y_test, y_hat, average = "weighted", scheme=IOB2, mode="strict")))

F1-score: 0.791


#  OPTIMIZATION

Implemented following sklearn-crfsuite documentation by Mikhail Korobov.

In [ ]:
import scipy
parameters = { 'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.5)}

In [ ]:
parameters

{'c1': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fc991efd450>,
 'c2': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fc9ae61da10>}

In [ ]:
from sklearn.metrics import make_scorer

def my_scorer1(y, y_pred):
  return f1_score(y, y_pred)

f1_scorer = make_scorer(my_scorer1)

In [ ]:
X, y = X_train, y_train

In [ ]:
from sklearn.model_selection import train_test_split
_, X_train, _, y_train = train_test_split(X, y, test_size = 0.3, random_state = 276)

In [ ]:
len(X_train)

9205

In [ ]:
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn.model_selection import RandomizedSearchCV


# search
rs = RandomizedSearchCV(CRF(), parameters,
                        cv=3,
                        verbose=100,
                        n_iter=50,
                        n_jobs= -1,
                        scoring = f1_scorer
                        )

In [ ]:
%%time
#rs.fit(X_train, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [ ]:
#rs.best_estimator_

In [ ]:
"""crf_optimized = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))"""

"crf_optimized = rs.best_estimator_\nprint('best params:', rs.best_params_)\nprint('best CV score:', rs.best_score_)\nprint('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))"

In [ ]:
"""crf_optimized = CRF(
    algorithm='lbfgs',
    c1= 0.27,
    c2= 0.24,
    max_iterations=100,
    all_possible_transitions=True,
)"""

"crf_optimized = CRF(\n    algorithm='lbfgs',\n    c1= 0.27,\n    c2= 0.24,\n    max_iterations=100,\n    all_possible_transitions=True,\n)"

In [ ]:
#crf_optimized.fit(X,y)

In [ ]:
#y_hat2 = crf_optimized.predict(X_test)

In [ ]:
# Save the model to disk
import joblib
filename = "/content/gdrive/MyDrive/BIO-NER-project/finalized_model.sav"
joblib.dump(crf_model, filename)

['/content/gdrive/MyDrive/BIO-NER-project/finalized_model.sav']

In [ ]:
# Load the model
loaded_model = joblib.load(filename)

In [ ]:
y_hat = loaded_model.predict(X_test)

# REPORT

In [ ]:
%%capture
# Install seqeval library for evaluation score
!pip3 install seqeval

In [ ]:
from seqeval.metrics import f1_score, classification_report

In [ ]:
from seqeval.scheme import IOB2
print('\033[1m' + "Classification report for BERT: ")
print('\033[0m')
print(classification_report(y_test, y_hat,scheme=IOB2, mode="strict"))

Classification report for BERT: 

              precision    recall  f1-score   support

    Chemical       0.86      0.73      0.79     25346

   micro avg       0.86      0.73      0.79     25346
   macro avg       0.86      0.73      0.79     25346
weighted avg       0.86      0.73      0.79     25346



In [ ]:
print("F1-score: {:0.3}".format(f1_score(y_test, y_hat, average = "micro", scheme=IOB2, mode="strict")))

F1-score: 0.791


In [ ]:
%%capture
!pip install nervaluate

In [ ]:
from nervaluate import Evaluator
golden_tags = ["Chemical"]
evaluator = Evaluator(y_test, y_hat, tags = golden_tags, loader="list")
results, results_by_tag = evaluator.evaluate()
import pandas as pd
res = pd.DataFrame.from_dict(results)

In [ ]:
res

,ent_type,partial,strict,exact
correct,20364.000000,18603.000000,18603.000000,18603.000000
incorrect,0.000000,0.000000,1761.000000,1761.000000
partial,0.000000,1761.000000,0.000000,0.000000
missed,4982.000000,4982.000000,4982.000000,4982.000000
spurious,1301.000000,1301.000000,1301.000000,1301.000000
possible,25346.000000,25346.000000,25346.000000,25346.000000
actual,21665.000000,21665.000000,21665.000000,21665.000000
precision,0.939949,0.899308,0.858666,0.858666
recall,0.803440,0.768701,0.733962,0.733962
f1,0.866350,0.828891,0.791432,0.791432


# DEBUGGING WITH ELI5

In [ ]:
%%capture
!pip3 install eli5

In [ ]:
import eli5
eli5.show_weights(crf_model, top=10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [ ]:
data_frame = eli5.format_as_dataframes(
            eli5.explain_weights_sklearn_crfsuite(crf_model))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [ ]:
data_frame

{'targets':         target                        feature    weight
 0            O                  wordID:saline  5.457855
 1            O              wordID:metabolite  5.345954
 2            O                    wordID:thin  5.313565
 3            O          wordID:polysaccharide  5.019502
 4            O                    wordID:gene  4.904046
 5            O                   word[:3]:Nan  4.628905
 6            O                    wordID:line  4.579896
 7            O                   wordID:alone  4.405060
 8            O                 wordID:protein  4.310939
 9            O                    wordID:Gene  4.260378
 10           O                  +2:wordID:TTO  4.209320
 11           O                   word[-2:]:ce  4.209209
 12           O         wordID:polycrystalline  4.188733
 13           O                   word[-2:]:ts  4.107797
 14           O             wordID:carotenoids  4.083030
 15           O                  word[-3:]:ble  4.042355
 16           O     

In [ ]:
from collections import Counter

def print_transitions(trans_features):
  for (label_from, label_to), weight in trans_features:
    print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))


In [ ]:
print("Top likely transitions:")
print_transitions(Counter(crf_model.transition_features_).most_common(20))

Top likely transitions:
O      -> O       1.143651
I-Chemical -> I-Chemical 0.768152
B-Chemical -> I-Chemical -0.001143
O      -> B-Chemical -0.058876
B-Chemical -> O       -0.676746
I-Chemical -> O       -1.726236
B-Chemical -> B-Chemical -2.198277
I-Chemical -> B-Chemical -3.279337
O      -> I-Chemical -12.662531


In [ ]:
print("\nTop unlikely transitions:")
print_transitions(Counter(crf_model.transition_features_).most_common()[-20:])


Top unlikely transitions:
O      -> O       1.143651
I-Chemical -> I-Chemical 0.768152
B-Chemical -> I-Chemical -0.001143
O      -> B-Chemical -0.058876
B-Chemical -> O       -0.676746
I-Chemical -> O       -1.726236
B-Chemical -> B-Chemical -2.198277
I-Chemical -> B-Chemical -3.279337
O      -> I-Chemical -12.662531


# COMPARE RESULTS

In [ ]:
# Compare results
i = 20
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*45)
for w, true, pred in zip(docs_test.sentences[i], y_test[i], y_hat[i]):
      print("{:15}:{:15} ({})".format(w, pred, true))

Word            Pred : (True)
Although       :O               (O)
genetic        :O               (O)
polymorphisms  :O               (O)
in             :O               (O)
the            :O               (O)
endothelial    :O               (O)
nitric         :B-Chemical      (B-Chemical)
oxide          :I-Chemical      (I-Chemical)
synthase       :O               (O)
(              :O               (O)
eNOS           :O               (O)
)              :O               (O)
gene           :O               (O)
may            :O               (O)
impair         :O               (O)
endogenous     :O               (O)
NO             :B-Chemical      (B-Chemical)
formation      :O               (O)
,              :O               (O)
there          :O               (O)
is             :O               (O)
little         :O               (O)
information    :O               (O)
about          :O               (O)
how            :O               (O)
eNOS           :O               (O)
polymor